In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
list1 = [["abc", "A"], ["bcd", "A"]]
list2 = [["ddd", "B"], ["rrr", "B"]]

df_sample = pd.DataFrame(list1 + list2)
df_sample

,0,1
0,abc,A
1,bcd,A
2,ddd,B
3,rrr,B


In [6]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [7]:
len(sentences)

5318

Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/

In [78]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_.lower()
                for token in text
                if not token.is_punct
                # simply calling token.is_stop mixes stop words in uppercase
                #and not token.is_stop and token.lemma_ != "-PRON-"]
                and not token.lemma_.lower() in stopwords.words('english')
                and not token.is_digit
                and token.lemma_ != "-PRON-"]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]


# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [79]:
# # tokenization practice
# for token in alice_doc[190:220]:
#     if token.lemma_.lower() in stopwords.words('english'):
#         print(token.lower_, " is stop word")
#     else:
#         print(token.lower_)
# check if any stop words are mixed
np.sum([word in stopwords.words('english') for word in alicewords])

0

In [87]:
common_words.remove("`")
common_words.remove("'s")

In [88]:
# Use custom tokenizer and lemmatizer
# a spacy span object is just a sentence nothing fancy
def lemmatizer_tokenize(spacy_span_input):
    words = [token.lemma_ for token in spacy_span_input
                if (
                    not token.is_digit and
                    not token.is_punct and 
                    not token.is_stop and 
                    token.lemma_ in common_words
                 )]
    return words    

In [90]:
# spacy pracitce
# sample_nlp = nlp("' `12 tinsae gizahcew @")
# for token in sample_nlp:
#     if( token.is_digit):
#         print(token)

In [92]:
# Optimizing the code by using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# these are stop words used by spacy
# stopwords.words('english'))
vectorizer = CountVectorizer(vocabulary=common_words, stop_words=stopwords.words('english'),
                             tokenizer=lemmatizer_tokenize, lowercase=False)
bow_features = pd.DataFrame(vectorizer.transform(sentences[0]).toarray(), columns = vectorizer.get_feature_names())
bow_features.head()

,abide,ability,able,abroad,absence,absent,absolute,absolutely,absurd,abuse,...,yes,yesterday,yet,yield,young,youth,zeal,zealand,zealous,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
vectorizer.get_feature_names()[:10]

['abide',
 'ability',
 'able',
 'abroad',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abuse']

In [74]:
# slow function
# # Creates a data frame with features for each word in our common word set.
# # Each value is the count of the times the word appears in each sentence.
# def bow_features(sentences, common_words):
    
#     # Scaffold the data frame and initialize counts to zero.
#     df = pd.DataFrame(columns=common_words)
#     df['text_sentence'] = sentences[0]
#     df['text_source'] = sentences[1]
#     df.loc[:, common_words] = 0
#     # Process each row, counting the occurrence of words in each sentence.
#     for i, sentence in enumerate(df['text_sentence'][0:1]):
        
#         # Convert the sentence to lemmas, then filter out punctuation,
#         # stop words, and uncommon words.
#         words = [token.lemma_
#                  for token in sentence
#                  if (
#                      not token.is_punct
#                      and not token.is_stop
#                      and token.lemma_ in common_words
#                  )]
        
#         # Populate the row with word counts.
#         for word in words:
#             df.loc[i, word] += 1
        
#         # This counter is just to make sure the kernel didn't hang.
#         if i % 500 == 0:
#             print("Processing row {}".format(i))
#         return df

In [94]:
len(common_words)

3012

In [95]:
bow_features.shape

(5318, 3012)

In [96]:
sentences.shape

(5318, 2)

In [97]:
bow_features.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [98]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = sentences[1]
X = bow_features.values

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\Tinsae\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9774294670846395

Test set score: 0.8632518796992481


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [99]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\Tinsae\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3012) (3190,)
Training set score: 0.94858934169279

Test set score: 0.8984962406015038


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [100]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8510971786833855

Test set score: 0.8529135338345865


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [21]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [22]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [101]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [102]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
#emma_bow = bow_features(emma_sentences, common_words)

# Optimizing the code by using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# these are stop words used by spacy
# stopwords.words('english'))
vectorizer = CountVectorizer(vocabulary=common_words, stop_words=stopwords.words('english'),
                             tokenizer=lemmatizer_tokenize, lowercase=False)
bow_features_ema = pd.DataFrame(vectorizer.transform(emma_sentences[0]).toarray(), columns = vectorizer.get_feature_names())
bow_features_ema.head()


,abide,ability,able,abroad,absence,absent,absolute,absolutely,absurd,abuse,...,yes,yesterday,yet,yield,young,youth,zeal,zealand,zealous,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
bow_features_ema.shape

(1669, 3012)

In [105]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
# we are using the train set on the code below
# I don't know why it says testset
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    bow_features_ema.values
    
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * bow_features_ema.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6882923191648024


col_0,Austen,Carroll
row_0,,
Austen,1553,116
Carroll,720,293


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [360]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(vocabulary=common_words, stop_words=stopwords.words('english'),
                             tokenizer=lemmatizer_tokenize, lowercase=False)
bow_features = pd.DataFrame(vectorizer.transform(sentences[0]).toarray(), columns = vectorizer.get_feature_names())
bow_features.head()

,abide,ability,able,abroad,absence,absent,absolute,absolutely,absurd,abuse,...,yes,yesterday,yet,yield,young,youth,zeal,zealand,zealous,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [361]:
bow_features.shape

(5318, 3012)

In [362]:
bow_features.head()

,abide,ability,able,abroad,absence,absent,absolute,absolutely,absurd,abuse,...,yes,yesterday,yet,yield,young,youth,zeal,zealand,zealous,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [363]:
bow_features["Num_of_Words"] = bow_features.sum(axis=1)
punc_count = []
cconcj_count = []
num_count = []
particle_count = []
the_poss = []
for sent in sentences[0]:
    pcount = 0
    cccount = 0
    ncount = 0
    pacount = 0
    for token in sent:
        the_poss.append(token.pos_)
        if(token.pos_ == 'PUNCT'):
            pcount += 1
        if(token.pos_ == "CCONJ"):
            cccount += 1
        if(token.pos_ == "NUM"):
            ncount += 1
        if(token.pos_ == "PART"):
            pacount += 1
    punc_count.append(pcount)
    cconcj_count.append(cccount)
    num_count.append(ncount)
    particle_count.append(pacount)

#print(punc_count[0:10])
bow_features["punctuation_count"] = punc_count
bow_features["conjuction_count"] = cconcj_count
bow_features["Numeric_count"] = num_count
bow_features["Particle_count"] = particle_count
#bow_features["previous_state_num_words"] = bow_features.Num_of_Words.shift(1).fillna(0)
#bow_features["next_state_num_words"] = bow_features.Num_of_Words.shift(-1).fillna(0)

Counter(the_poss)

Counter({'PROPN': 6311,
         'VERB': 24307,
         'PART': 3504,
         'ADV': 10138,
         'ADJ': 11032,
         'ADP': 13855,
         'NOUN': 16289,
         'DET': 9717,
         'PUNCT': 21312,
         'CCONJ': 4965,
         'PRON': 10185,
         'INTJ': 374,
         'NUM': 731,
         'X': 2})

In [364]:
# split into train and test set
Y = sentences[1]
X = bow_features.values

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [365]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import Normalizer
# scaler = StandardScaler()
# normalizer = Normalizer()
# X_train = normalizer.fit_transform(X_train)
# X_test = normalizer.fit_transform(X_test)

In [389]:
lr = LogisticRegression(C=1.5)
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\Tinsae\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3017) (3190,)
Training set score: 0.9567398119122257

Test set score: 0.9046052631578947


In [367]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
train = lsvc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lsvc.score(X_train, y_train))
print('\nTest set score:', lsvc.score(X_test, y_test))

C:\Users\Tinsae\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


(3190, 3017) (3190,)
Training set score: 0.9780564263322884

Test set score: 0.8872180451127819


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [391]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
#emma_bow = bow_features(emma_sentences, common_words)

# Optimizing the code by using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# these are stop words used by spacy
# stopwords.words('english'))
vectorizer = CountVectorizer(vocabulary=common_words, stop_words=stopwords.words('english'),
                             tokenizer=lemmatizer_tokenize, lowercase=False)
bow_features_ema = pd.DataFrame(vectorizer.transform(emma_sentences[0]).toarray(), columns = vectorizer.get_feature_names())
bow_features_ema.head()


,abide,ability,able,abroad,absence,absent,absolute,absolutely,absurd,abuse,...,yes,yesterday,yet,yield,young,youth,zeal,zealand,zealous,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [395]:
bow_features_ema["Num_of_Words"] = bow_features_ema.sum(axis=1)
punc_count = []
cconcj_count = []
num_count = []
particle_count = []
the_poss = []
for sent in emma_sentences[0]:
    pcount = 0
    cccount = 0
    ncount = 0
    pacount = 0
    for token in sent:
        the_poss.append(token.pos_)
        if(token.pos_ == 'PUNCT'):
            pcount += 1
        if(token.pos_ == "CCONJ"):
            cccount += 1
        if(token.pos_ == "NUM"):
            ncount += 1
        if(token.pos_ == "PART"):
            pacount += 1
    punc_count.append(pcount)
    cconcj_count.append(cccount)
    num_count.append(ncount)
    particle_count.append(pacount)

#print(punc_count[0:10])
bow_features_ema["punctuation_count"] = punc_count
bow_features_ema["conjuction_count"] = cconcj_count
bow_features_ema["Numeric_count"] = num_count
bow_features_ema["Particle_count"] = particle_count
#bow_features["previous_state_num_words"] = bow_features.Num_of_Words.shift(1).fillna(0)
#bow_features["next_state_num_words"] = bow_features.Num_of_Words.shift(-1).fillna(0)

Counter(the_poss)

Counter({'PROPN': 1580,
         'PUNCT': 5422,
         'ADJ': 3473,
         'CCONJ': 1463,
         'ADP': 3649,
         'DET': 2511,
         'NOUN': 4647,
         'VERB': 6792,
         'PART': 915,
         'ADV': 3045,
         'NUM': 188,
         'PRON': 3052,
         'INTJ': 131})

In [396]:
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
# we are using the train set on the code below
# I don't know why it says testset
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    bow_features_ema.values
    
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * bow_features_ema.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7248322147651006


col_0,Austen,Carroll
row_0,,
Austen,1640,29
Carroll,709,304


In [ ]:
# Better than previous model